## Importing Libraries

In [10]:
!pip install import_ipynb

In [11]:
from scipy.io import loadmat
import numpy as np
import import_ipynb
from Model import neural_network
from RandInitialize import initialise
from Prediction import predict
from scipy.optimize import minimize

## Load Datasets

In [12]:
# Loading mat file
data = loadmat('mnist-original.mat')

## Data Analysis

In [13]:
# Extracting features from mat file
X = data['data']
X = X.transpose()

In [14]:
# Normalizing the data
X = X / 255

# Extracting labels from mat file
y = data['label']
y = y.flatten()

In [15]:
# Splitting data into training set with 60,000 examples
X_train = X[:60000, :]
y_train = y[:60000]

# Splitting data into testing set with 10,000 examples
X_test = X[60000:, :]
y_test = y[60000:]

In [16]:
m = X.shape[0]
input_layer_size = 784 
hidden_layer_size = 100
num_labels = 10 

In [17]:
# Randomly initialising Thetas
initial_Theta1 = initialise(hidden_layer_size, input_layer_size)
initial_Theta2 = initialise(num_labels, hidden_layer_size)

In [18]:
initial_nn_params = np.concatenate((initial_Theta1.flatten(), initial_Theta2.flatten()))
maxiter = 100
lambda_reg = 0.1 # To avoid overfitting
myargs = (input_layer_size, hidden_layer_size, num_labels, X_train, y_train, lambda_reg)

In [19]:
results = minimize(neural_network, x0=initial_nn_params, args=myargs,
        options={'disp': True, 'maxiter': maxiter}, method="L-BFGS-B", jac=True)

nn_params = results["x"] 

In [20]:
# Weights are split back to Theta1, Theta2
Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)], (
                    hidden_layer_size, input_layer_size + 1)) # shape = (100, 785)
Theta2 = np.reshape(nn_params[hidden_layer_size * (input_layer_size + 1):],
                    (num_labels, hidden_layer_size + 1)) # shape = (10, 101)

## Test the Model

In [21]:
# Checking test set accuracy of our model
pred = predict(Theta1, Theta2, X_test)
print('Test Set Accuracy: {:f}'.format((np.mean(pred == y_test) * 100)))

# Checking train set accuracy of our model
pred = predict(Theta1, Theta2, X_train)
print('Training Set Accuracy: {:f}'.format((np.mean(pred == y_train) * 100)))

Test Set Accuracy: 97.320000
Training Set Accuracy: 99.416667


In [22]:
# Evaluating precision of our model
true_positive = 0
for i in range(len(pred)):
    if pred[i] == y_train[i]:
        true_positive += 1
false_positive = len(y_train) - true_positive
print('Precision =', true_positive/(true_positive + false_positive))

Precision = 0.9941666666666666


In [23]:
# Saving Thetas in .txt file
np.savetxt('Theta1.txt', Theta1, delimiter=' ')
np.savetxt('Theta2.txt', Theta2, delimiter=' ')